# NYC Weather Data Cleaning Pipeline

This notebook cleans and prepares NYC weather data (2016-2022) for merging with Citi Bike trip data. The pipeline includes:
- Downloading data from Kaggle
- Standardizing column names
- Filtering for year 2018
- Handling missing values
- Creating derived features
- Merging with Citi Bike data

---

## Step 1: Import Libraries and Download Dataset

Import necessary Python libraries (`kagglehub`, `pandas`, `os`) and download the NYC Weather dataset (2016-2022) from Kaggle using the `kagglehub` API.

In [1]:
import kagglehub
import pandas as pd
import os

# Download latest version
path = kagglehub.dataset_download("aadimator/nyc-weather-2016-to-2022")

print("Path to dataset files:", path)

Path to dataset files: /Users/hstro/.cache/kagglehub/datasets/aadimator/nyc-weather-2016-to-2022/versions/1


## Step 2: Load the Weather CSV File

Load the downloaded CSV file into a pandas DataFrame. This cell:
- Lists all CSV files in the download directory
- Loads the weather data file
- Displays the dataset shape, column names, and first few rows for initial inspection

In [2]:
# Load the weather data
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
print(f"Found CSV files: {csv_files}")

# Load the data (assuming there's one main CSV file)
weather_file = os.path.join(path, csv_files[0])
df = pd.read_csv(weather_file)

print(f"\nOriginal dataset shape: {df.shape}")
print(f"\nOriginal column names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

Found CSV files: ['NYC_Weather_2016_2022.csv']

Original dataset shape: (59760, 10)

Original column names:
['time', 'temperature_2m (°C)', 'precipitation (mm)', 'rain (mm)', 'cloudcover (%)', 'cloudcover_low (%)', 'cloudcover_mid (%)', 'cloudcover_high (%)', 'windspeed_10m (km/h)', 'winddirection_10m (°)']

First few rows:


,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2016-01-01T00:00,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0
1,2016-01-01T01:00,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0
2,2016-01-01T02:00,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0
3,2016-01-01T03:00,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0
4,2016-01-01T04:00,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0


## Step 3: Standardize Column Names

Clean up column names by:
- Converting all names to lowercase
- Replacing spaces with underscores

This ensures consistent naming conventions throughout the analysis.

In [3]:
# Step 1: Standardize column names (lowercase and replace spaces with underscores)
df.columns = df.columns.str.lower().str.replace(' ', '_')

print("Standardized column names:")
print(df.columns.tolist())
df.head()

Standardized column names:
['time', 'temperature_2m_(°c)', 'precipitation_(mm)', 'rain_(mm)', 'cloudcover_(%)', 'cloudcover_low_(%)', 'cloudcover_mid_(%)', 'cloudcover_high_(%)', 'windspeed_10m_(km/h)', 'winddirection_10m_(°)']


,time,temperature_2m_(°c),precipitation_(mm),rain_(mm),cloudcover_(%),cloudcover_low_(%),cloudcover_mid_(%),cloudcover_high_(%),windspeed_10m_(km/h),winddirection_10m_(°)
0,2016-01-01T00:00,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0
1,2016-01-01T01:00,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0
2,2016-01-01T02:00,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0
3,2016-01-01T03:00,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0
4,2016-01-01T04:00,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0


## Step 4: Filter for Year 2018

Since our Citi Bike data is from 2018, we filter the weather dataset to keep only records from year 2018. This cell:
- Identifies the date column (`time`)
- Converts it to datetime format
- Filters rows to retain only 2018 data (reduces from ~60,000 to ~8,760 rows - one hourly reading per hour of the year)

In [4]:
# Step 2: Keep only rows from year 2018
# First, identify the date column
date_columns = [col for col in df.columns if 'date' in col or 'time' in col]
print(f"Date-related columns found: {date_columns}")

# Convert date column to datetime
if date_columns:
    date_col = date_columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Filter for year 2018
    initial_rows = len(df)
    df = df[df[date_col].dt.year == 2018].copy()
    print(f"\nFiltered from {initial_rows} rows to {len(df)} rows (year 2018 only)")
else:
    print("Warning: No date column found!")

df.head()

Date-related columns found: ['time']

Filtered from 59760 rows to 8760 rows (year 2018 only)


,time,temperature_2m_(°c),precipitation_(mm),rain_(mm),cloudcover_(%),cloudcover_low_(%),cloudcover_mid_(%),cloudcover_high_(%),windspeed_10m_(km/h),winddirection_10m_(°)
17544,2018-01-01 00:00:00,-11.1,0.0,0.0,0.0,0.0,0.0,0.0,9.4,302.0
17545,2018-01-01 01:00:00,-11.5,0.0,0.0,0.0,0.0,0.0,0.0,10.1,297.0
17546,2018-01-01 02:00:00,-11.8,0.0,0.0,0.0,0.0,0.0,0.0,12.3,302.0
17547,2018-01-01 03:00:00,-12.2,0.0,0.0,0.0,0.0,0.0,0.0,13.9,307.0
17548,2018-01-01 04:00:00,-12.2,0.0,0.0,0.0,0.0,0.0,0.0,14.3,309.0


## Step 5: Identify Weather Columns and Check Missing Values

Identify the key weather columns needed for analysis:
- **Temperature**: `temperature_2m_(°c)`
- **Precipitation**: `precipitation_(mm)`
- **Wind Speed**: `windspeed_10m_(km/h)`

Check for missing values in these essential columns before proceeding with cleaning.

In [5]:
# Step 3: Identify weather columns and check for missing values
# Find temperature, precipitation, and wind columns
temp_col = None
precip_col = None
wind_col = None

for col in df.columns:
    if 'temp' in col and not 'min' in col and not 'max' in col:
        temp_col = col
    elif 'precip' in col:
        precip_col = col
    elif 'wind' in col and 'speed' in col:
        wind_col = col
    elif 'wind' in col and wind_col is None:
        wind_col = col

print("Identified weather columns:")
print(f"  Temperature: {temp_col}")
print(f"  Precipitation: {precip_col}")
print(f"  Wind: {wind_col}")

# Display missing values
print("\nMissing values before cleaning:")
if temp_col:
    print(f"  {temp_col}: {df[temp_col].isna().sum()}")
if precip_col:
    print(f"  {precip_col}: {df[precip_col].isna().sum()}")
if wind_col:
    print(f"  {wind_col}: {df[wind_col].isna().sum()}")

Identified weather columns:
  Temperature: temperature_2m_(°c)
  Precipitation: precipitation_(mm)
  Wind: windspeed_10m_(km/h)

Missing values before cleaning:
  temperature_2m_(°c): 0
  precipitation_(mm): 0
  windspeed_10m_(km/h): 0


## Step 6: Remove Rows with Missing Weather Values

Remove any rows that have missing values in the essential weather columns (temperature, precipitation, wind speed). This ensures data quality for the analysis. Also display the valid range of values for each weather variable.

In [6]:
# Step 3 (continued): Remove rows with missing essential weather values
initial_rows = len(df)

# Drop rows with missing values in essential columns
essential_cols = [col for col in [temp_col, precip_col, wind_col] if col is not None]
df = df.dropna(subset=essential_cols)

print(f"Removed {initial_rows - len(df)} rows with missing weather values")
print(f"Remaining rows: {len(df)}")

# Check for and remove obviously invalid values (e.g., extreme outliers)
if temp_col:
    print(f"\nTemperature range: {df[temp_col].min()} to {df[temp_col].max()}")
if precip_col:
    print(f"Precipitation range: {df[precip_col].min()} to {df[precip_col].max()}")
if wind_col:
    print(f"Wind range: {df[wind_col].min()} to {df[wind_col].max()}")

Removed 0 rows with missing weather values
Remaining rows: 8760

Temperature range: -18.3 to 34.5
Precipitation range: 0.0 to 6.8
Wind range: 0.4 to 45.3


## Step 7: Temperature Unit Check

Verify that temperature is in Celsius. The check compares the mean temperature against typical NYC ranges:
- **Celsius**: Mean ~12°C (range: -18°C to 35°C)
- **Fahrenheit**: Mean ~54°F (range: 14°F to 95°F)

If the mean is above 40, it's likely in Fahrenheit and needs conversion.

In [7]:
# Step 4: Convert temperature to Celsius if needed
if temp_col:
    # Check if temperature is in Fahrenheit
    # NYC typical temperature range:
    # - Celsius: -10 to 35°C
    # - Fahrenheit: 14 to 95°F
    
    temp_mean = df[temp_col].mean()
    temp_max = df[temp_col].max()
    
    print(f"\nTemperature statistics:")
    print(f"  Mean: {temp_mean:.2f}")
    print(f"  Max: {temp_max:.2f}")
    print(f"  Min: {df[temp_col].min():.2f}")
    
    # If mean temperature is above 40, likely Fahrenheit
    if temp_mean > 40:
        print("\n✓ Detected Fahrenheit - Converting to Celsius...")
        df[temp_col] = (df[temp_col] - 32) * 5/9
        print(f"  New mean temperature: {df[temp_col].mean():.2f}°C")
        print(f"  New range: {df[temp_col].min():.2f}°C to {df[temp_col].max():.2f}°C")
    else:
        print("\n✓ Temperature already in Celsius")
else:
    print("Warning: No temperature column found for conversion")


Temperature statistics:
  Mean: 12.76
  Max: 34.50
  Min: -18.30

✓ Temperature already in Celsius


## Step 8: Clean Precipitation Data

Process the precipitation column to create a standardized `precip_mm` column. This step handles both:
- **Numeric data**: Already in mm, create standardized column
- **Categorical data**: Create binary `rain_flag` indicator

The output shows precipitation statistics including range, mean, and number of days with precipitation.

In [8]:
# Step 5: Clean precipitation column
print("="*60)
print("PRECIPITATION CLEANING")
print("="*60)

if precip_col:
    print(f"\nAnalyzing precipitation column: {precip_col}")
    print(f"Data type: {df[precip_col].dtype}")
    print(f"\nSample values:")
    print(df[precip_col].head(10))
    
    # Check if precipitation is numeric or categorical
    if pd.api.types.is_numeric_dtype(df[precip_col]):
        # Case 5a: Numeric precipitation
        print("\n✓ Detected NUMERIC precipitation data")
        
        # Standardize to millimeters
        df["precip_mm"] = pd.to_numeric(df[precip_col], errors="coerce").fillna(0)
        
        print(f"  Created 'precip_mm' column")
        print(f"  Range: {df['precip_mm'].min():.2f} to {df['precip_mm'].max():.2f} mm")
        print(f"  Mean: {df['precip_mm'].mean():.2f} mm")
        print(f"  Days with precipitation > 0: {(df['precip_mm'] > 0).sum()}")
        
    else:
        # Case 5b: Categorical precipitation
        print("\n✓ Detected CATEGORICAL precipitation data")
        print(f"\nUnique values: {df[precip_col].unique()}")
        
        # Create rain flag
        df["rain_flag"] = df[precip_col].astype(str).str.lower().str.contains("rain", na=False).astype(int)
        print(f"  Created 'rain_flag' column")
        print(f"  Days with rain: {df['rain_flag'].sum()}")
        
        # Create snow flag if snow is present
        if df[precip_col].astype(str).str.lower().str.contains("snow", na=False).any():
            df["snow_flag"] = df[precip_col].astype(str).str.lower().str.contains("snow", na=False).astype(int)
            print(f"  Created 'snow_flag' column")
            print(f"  Days with snow: {df['snow_flag'].sum()}")
        
        # Also check for weather_condition column if it exists
        weather_cond_cols = [col for col in df.columns if 'weather' in col and 'condition' in col]
        if weather_cond_cols:
            weather_col = weather_cond_cols[0]
            print(f"\n  Found additional weather condition column: {weather_col}")
            
            # Create additional flags from weather_condition if not already created
            if 'rain_flag' not in df.columns or df['rain_flag'].sum() == 0:
                df["rain_flag"] = df[weather_col].astype(str).str.lower().str.contains("rain", na=False).astype(int)
                print(f"  Updated 'rain_flag' from {weather_col}")
                print(f"  Days with rain: {df['rain_flag'].sum()}")
            
            if 'snow_flag' not in df.columns and df[weather_col].astype(str).str.lower().str.contains("snow", na=False).any():
                df["snow_flag"] = df[weather_col].astype(str).str.lower().str.contains("snow", na=False).astype(int)
                print(f"  Created 'snow_flag' from {weather_col}")
                print(f"  Days with snow: {df['snow_flag'].sum()}")
else:
    print("\nWarning: No precipitation column found!")

print("\n" + "="*60)

PRECIPITATION CLEANING

Analyzing precipitation column: precipitation_(mm)
Data type: float64

Sample values:
17544    0.0
17545    0.0
17546    0.0
17547    0.0
17548    0.0
17549    0.0
17550    0.0
17551    0.0
17552    0.0
17553    0.0
Name: precipitation_(mm), dtype: float64

✓ Detected NUMERIC precipitation data
  Created 'precip_mm' column
  Range: 0.00 to 6.80 mm
  Mean: 0.16 mm
  Days with precipitation > 0: 1568



## Step 9: Clean Wind Speed Data

Process wind speed data and standardize to km/h. The cell:
- Converts wind speed to numeric format
- Checks if data is in mph (mean < 25) or km/h (mean ≥ 25)
- Converts from mph to km/h if necessary (multiply by 1.60934)
- Creates standardized `wind_kmh` column

In [9]:
# Step 7: Clean wind speed
print("="*60)
print("WIND SPEED CLEANING")
print("="*60)

if wind_col:
    print(f"\nAnalyzing wind column: {wind_col}")
    print(f"Data type: {df[wind_col].dtype}")
    
    # Ensure wind speed is numeric
    df["wind_speed"] = pd.to_numeric(df[wind_col], errors="coerce")
    
    # Remove any NaN values that resulted from conversion
    nan_count = df["wind_speed"].isna().sum()
    if nan_count > 0:
        print(f"  Converted {nan_count} non-numeric values to NaN")
        df = df.dropna(subset=["wind_speed"])
        print(f"  Removed rows with invalid wind speed")
    
    # Check units and convert to km/h if needed
    wind_mean = df["wind_speed"].mean()
    wind_max = df["wind_speed"].max()
    
    print(f"\nWind speed statistics:")
    print(f"  Mean: {wind_mean:.2f}")
    print(f"  Max: {wind_max:.2f}")
    print(f"  Min: {df['wind_speed'].min():.2f}")
    
    # NYC typical wind speed:
    # - mph: average around 10-15 mph, rarely exceeds 40 mph
    # - km/h: average around 16-24 km/h, rarely exceeds 65 km/h
    
    if wind_mean < 25:  # Likely in mph
        print("\n✓ Detected mph - Converting to km/h...")
        df["wind_kmh"] = df["wind_speed"] * 1.60934
        print(f"  Created 'wind_kmh' column")
        print(f"  New mean: {df['wind_kmh'].mean():.2f} km/h")
        print(f"  New range: {df['wind_kmh'].min():.2f} to {df['wind_kmh'].max():.2f} km/h")
    else:
        print("\n✓ Already in km/h - Renaming column...")
        df.rename(columns={"wind_speed": "wind_kmh"}, inplace=True)
        print(f"  Renamed to 'wind_kmh'")
        print(f"  Range: {df['wind_kmh'].min():.2f} to {df['wind_kmh'].max():.2f} km/h")
else:
    print("\nWarning: No wind column found!")

print("\n" + "="*60)

WIND SPEED CLEANING

Analyzing wind column: windspeed_10m_(km/h)
Data type: float64

Wind speed statistics:
  Mean: 11.46
  Max: 45.30
  Min: 0.40

✓ Detected mph - Converting to km/h...
  Created 'wind_kmh' column
  New mean: 18.45 km/h
  New range: 0.64 to 72.90 km/h



## Step 10: Create Derived Feature Columns

Create additional features useful for analysis:

1. **`temp_bin`**: Temperature categories (<0°C, 0-10°C, 10-20°C, 20-30°C, >30°C)
2. **`rain_flag`**: Binary indicator (1 = precipitation > 0mm, 0 = no rain)
3. **`month`**: Extracted month number (1-12)
4. **`season`**: Categorical season (Winter, Spring, Summer, Autumn)

In [10]:
# Step 8: Create useful derived columns
print("="*60)
print("CREATING DERIVED COLUMNS")
print("="*60)

# 1. Temperature bins
if temp_col:
    # Use the temperature column (already converted to Celsius)
    temp_column_to_use = temp_col
    
    df["temp_bin"] = pd.cut(
        df[temp_column_to_use],
        bins=[-50, 0, 10, 20, 30, 50],
        labels=["<0°C", "0–10°C", "10–20°C", "20–30°C", ">30°C"]
    )
    
    print("\n✓ Created 'temp_bin' column")
    print(f"  Temperature distribution:")
    print(df["temp_bin"].value_counts().sort_index())
else:
    print("\nWarning: Cannot create temperature bins - no temperature column found")

# 2. Rain flag from numeric precipitation
if "precip_mm" in df.columns:
    # Create rain flag from numeric precipitation
    df["rain_flag"] = (df["precip_mm"] > 0).astype(int)
    print(f"\n✓ Created 'rain_flag' from numeric precipitation")
    print(f"  Days with rain (precip > 0 mm): {df['rain_flag'].sum()}")
    print(f"  Days without rain: {(df['rain_flag'] == 0).sum()}")
elif "rain_flag" in df.columns:
    print(f"\n✓ Rain flag already exists (from categorical data)")
    print(f"  Days with rain: {df['rain_flag'].sum()}")
else:
    print("\nWarning: No precipitation data available to create rain flag")

# 3. Month and Season
if date_columns:
    date_col = date_columns[0]
    
    # Extract month
    df["month"] = df[date_col].dt.month
    print(f"\n✓ Created 'month' column")
    
    # Create season
    df["season"] = pd.cut(
        df["month"],
        bins=[0, 3, 6, 9, 12],
        labels=["Winter", "Spring", "Summer", "Autumn"],
        include_lowest=True
    )
    
    print(f"✓ Created 'season' column")
    print(f"  Seasonal distribution:")
    print(df["season"].value_counts().sort_index())
else:
    print("\nWarning: Cannot create month/season - no date column found")

print("\n" + "="*60)
print("\nNew derived columns added:")
derived_cols = ["temp_bin", "rain_flag", "month", "season"]
existing_derived = [col for col in derived_cols if col in df.columns]
for col in existing_derived:
    print(f"  - {col}")

CREATING DERIVED COLUMNS

✓ Created 'temp_bin' column
  Temperature distribution:
temp_bin
<0°C        842
0–10°C     3052
10–20°C    2092
20–30°C    2596
>30°C       178
Name: count, dtype: int64

✓ Created 'rain_flag' from numeric precipitation
  Days with rain (precip > 0 mm): 1568
  Days without rain: 7192

✓ Created 'month' column
✓ Created 'season' column
  Seasonal distribution:
season
Winter    2160
Spring    2184
Summer    2208
Autumn    2208
Name: count, dtype: int64


New derived columns added:
  - temp_bin
  - rain_flag
  - month
  - season


## Step 11: Prepare for Citi Bike Merge

Prepare merge keys to join weather data with Citi Bike trip data. Since weather data is hourly:
- **`date`**: Extract date in YYYY-MM-DD format
- **`hour`**: Extract hour (0-23)

These columns will serve as merge keys to match each bike trip with the corresponding weather conditions.

In [11]:
# Step 9: Ensure merge compatibility with Citi Bike trips
print("="*60)
print("PREPARE FOR CITI BIKE MERGE")
print("="*60)

if date_columns:
    date_col = date_columns[0]
    
    # Check if we have datetime (with time) or just date
    # Look for columns that might contain time information
    datetime_cols = [col for col in df.columns if 'datetime' in col or 'time' in col]
    has_hourly_data = len(datetime_cols) > 0
    
    print(f"\nDate/time columns found: {date_columns}")
    
    if has_hourly_data:
        # Case: Dataset has hourly readings
        print(f"\n✓ Detected HOURLY data in column: {date_col}")
        
        # Ensure datetime format
        if not pd.api.types.is_datetime64_any_dtype(df[date_col]):
            df[date_col] = pd.to_datetime(df[date_col])
        
        # Extract hour
        df["hour"] = df[date_col].dt.hour
        print(f"  Created 'hour' column (0-23)")
        print(f"  Hour distribution (sample):\")\n{df['hour'].value_counts().sort_index().head(10)}")
        
        # Extract date in YYYY-MM-DD format
        df["date"] = df[date_col].dt.date
        print(f"  Created 'date' column in YYYY-MM-DD format")
        
    else:
        # Case: Dataset has daily data only
        print(f"\n✓ Detected DAILY data in column: {date_col}")
        
        # Extract date in YYYY-MM-DD format (convert datetime to date)
        if pd.api.types.is_datetime64_any_dtype(df[date_col]):
            df["date"] = df[date_col].dt.date
            print(f"  Created 'date' column in YYYY-MM-DD format")
        else:
            # If it's already a date, just rename or copy
            if date_col != "date":
                df["date"] = pd.to_datetime(df[date_col]).dt.date
                print(f"  Created 'date' column in YYYY-MM-DD format")
            else:
                print(f"  Date column already exists in correct format")
    
    # Show sample of date format
    print(f"\n  Sample dates:")
    print(f"  {df['date'].head(10).tolist()}")
    
    if "hour" in df.columns:
        print(f"\n  Sample date-hour combinations:")
        print(df[["date", "hour"]].head(10))
    
    print(f"\n✓ Data is now ready to merge with Citi Bike trips!")
    print(f"  Merge key: 'date' (YYYY-MM-DD)")
    if "hour" in df.columns:
        print(f"  Optional merge key: 'date' + 'hour' for hourly analysis")
    
else:
    print("\nWarning: No date column found - cannot prepare for merge!")

print("\n" + "="*60)

PREPARE FOR CITI BIKE MERGE

Date/time columns found: ['time']

✓ Detected HOURLY data in column: time
  Created 'hour' column (0-23)
  Hour distribution (sample):")
hour
0    365
1    365
2    365
3    365
4    365
5    365
6    365
7    365
8    365
9    365
Name: count, dtype: int64
  Created 'date' column in YYYY-MM-DD format

  Sample dates:
  [datetime.date(2018, 1, 1), datetime.date(2018, 1, 1), datetime.date(2018, 1, 1), datetime.date(2018, 1, 1), datetime.date(2018, 1, 1), datetime.date(2018, 1, 1), datetime.date(2018, 1, 1), datetime.date(2018, 1, 1), datetime.date(2018, 1, 1), datetime.date(2018, 1, 1)]

  Sample date-hour combinations:
             date  hour
17544  2018-01-01     0
17545  2018-01-01     1
17546  2018-01-01     2
17547  2018-01-01     3
17548  2018-01-01     4
17549  2018-01-01     5
17550  2018-01-01     6
17551  2018-01-01     7
17552  2018-01-01     8
17553  2018-01-01     9

✓ Data is now ready to merge with Citi Bike trips!
  Merge key: 'date' (YYYY-MM

## Step 12: Data Cleaning Summary

Display a comprehensive summary of the cleaned weather dataset including:
- Final shape (rows × columns)
- All column names
- Missing value counts per column
- Data types
- Preview of cleaned data

In [12]:
# Final summary of cleaned data
print("="*60)
print("DATA CLEANING SUMMARY")
print("="*60)
print(f"\nFinal dataset shape: {df.shape}")
print(f"\nColumn names after cleaning:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

print(f"\nMissing values in final dataset:")
print(df.isna().sum())

print(f"\nData types:")
print(df.dtypes)

print(f"\nFirst few rows of cleaned data:")
df.head(10)

DATA CLEANING SUMMARY

Final dataset shape: (8760, 19)

Column names after cleaning:
  1. time
  2. temperature_2m_(°c)
  3. precipitation_(mm)
  4. rain_(mm)
  5. cloudcover_(%)
  6. cloudcover_low_(%)
  7. cloudcover_mid_(%)
  8. cloudcover_high_(%)
  9. windspeed_10m_(km/h)
  10. winddirection_10m_(°)
  11. precip_mm
  12. wind_speed
  13. wind_kmh
  14. temp_bin
  15. rain_flag
  16. month
  17. season
  18. hour
  19. date

Missing values in final dataset:
time                     0
temperature_2m_(°c)      0
precipitation_(mm)       0
rain_(mm)                0
cloudcover_(%)           0
cloudcover_low_(%)       0
cloudcover_mid_(%)       0
cloudcover_high_(%)      0
windspeed_10m_(km/h)     0
winddirection_10m_(°)    0
precip_mm                0
wind_speed               0
wind_kmh                 0
temp_bin                 0
rain_flag                0
month                    0
season                   0
hour                     0
date                     0
dtype: int64

Data ty

,time,temperature_2m_(°c),precipitation_(mm),rain_(mm),cloudcover_(%),cloudcover_low_(%),cloudcover_mid_(%),cloudcover_high_(%),windspeed_10m_(km/h),winddirection_10m_(°),precip_mm,wind_speed,wind_kmh,temp_bin,rain_flag,month,season,hour,date
17544,2018-01-01 00:00:00,-11.1,0.0,0.0,0.0,0.0,0.0,0.0,9.4,302.0,0.0,9.4,15.127796,<0°C,0,1,Winter,0,2018-01-01
17545,2018-01-01 01:00:00,-11.5,0.0,0.0,0.0,0.0,0.0,0.0,10.1,297.0,0.0,10.1,16.254334,<0°C,0,1,Winter,1,2018-01-01
17546,2018-01-01 02:00:00,-11.8,0.0,0.0,0.0,0.0,0.0,0.0,12.3,302.0,0.0,12.3,19.794882,<0°C,0,1,Winter,2,2018-01-01
17547,2018-01-01 03:00:00,-12.2,0.0,0.0,0.0,0.0,0.0,0.0,13.9,307.0,0.0,13.9,22.369826,<0°C,0,1,Winter,3,2018-01-01
17548,2018-01-01 04:00:00,-12.2,0.0,0.0,0.0,0.0,0.0,0.0,14.3,309.0,0.0,14.3,23.013562,<0°C,0,1,Winter,4,2018-01-01
17549,2018-01-01 05:00:00,-12.3,0.0,0.0,0.0,0.0,0.0,0.0,14.1,310.0,0.0,14.1,22.691694,<0°C,0,1,Winter,5,2018-01-01
17550,2018-01-01 06:00:00,-12.4,0.0,0.0,0.0,0.0,0.0,0.0,13.8,311.0,0.0,13.8,22.208892,<0°C,0,1,Winter,6,2018-01-01
17551,2018-01-01 07:00:00,-12.4,0.0,0.0,0.0,0.0,0.0,0.0,12.8,310.0,0.0,12.8,20.599552,<0°C,0,1,Winter,7,2018-01-01
17552,2018-01-01 08:00:00,-12.5,0.0,0.0,0.0,0.0,0.0,0.0,11.3,307.0,0.0,11.3,18.185542,<0°C,0,1,Winter,8,2018-01-01
17553,2018-01-01 09:00:00,-13.0,0.0,0.0,0.0,0.0,0.0,0.0,11.1,306.0,0.0,11.1,17.863674,<0°C,0,1,Winter,9,2018-01-01


## Step 13: Save Cleaned Weather Data

Export the cleaned weather dataset to a CSV file (`cleaned_weather_data.csv`). This file contains:
- 8,760 hourly records for 2018
- 19 columns including original and derived features
- Ready for merging with Citi Bike trip data

In [13]:
# Save the cleaned data to CSV file
print("="*60)
print("SAVING CLEANED DATA")
print("="*60)

output_filename = "cleaned_weather_data.csv"

# Save to CSV
df.to_csv(output_filename, index=False)

print(f"\n✓ Cleaned weather data saved successfully!")
print(f"  Filename: {output_filename}")
print(f"  Location: {os.getcwd()}")
print(f"  Rows: {len(df)}")
print(f"  Columns: {len(df.columns)}")
print(f"  File size: {os.path.getsize(output_filename) / 1024:.2f} KB")

print(f"\n  Columns saved:")
for i, col in enumerate(df.columns, 1):
    print(f"    {i}. {col}")

print("\n" + "="*60)
print("Data cleaning complete! Ready to merge with Citi Bike data.")
print("="*60)

SAVING CLEANED DATA

✓ Cleaned weather data saved successfully!
  Filename: cleaned_weather_data.csv
  Location: /Users/hstro/Documents/DTU/1st Semester/Introduction to business analytics/Project/Part 2
  Rows: 8760
  Columns: 19
  File size: 1006.54 KB

  Columns saved:
    1. time
    2. temperature_2m_(°c)
    3. precipitation_(mm)
    4. rain_(mm)
    5. cloudcover_(%)
    6. cloudcover_low_(%)
    7. cloudcover_mid_(%)
    8. cloudcover_high_(%)
    9. windspeed_10m_(km/h)
    10. winddirection_10m_(°)
    11. precip_mm
    12. wind_speed
    13. wind_kmh
    14. temp_bin
    15. rain_flag
    16. month
    17. season
    18. hour
    19. date

Data cleaning complete! Ready to merge with Citi Bike data.


## Sanity Check - Merged Dataset

Now let's perform sanity checks on the merged dataset to verify the merge was successful and data quality is maintained.

## Load and Prepare Citi Bike Data for Merging

Now we'll load the Citi Bike trip data and prepare it for merging with the weather data.

### Load All Citi Bike CSV Files

Load all 2018 Citi Bike trip data CSV files from the local directory. This cell:
- Finds all CSV files containing "2018" in the filename
- Loads each file into a DataFrame
- Concatenates all DataFrames into a single dataset (~18.8 million trips)

In [14]:
# Load Citi Bike trip data
print("="*60)
print("LOADING CITI BIKE DATA")
print("="*60)

# Path to Citi Bike data directory
citibike_path = "../2018-citibike-tripdata/"

# Get all CSV files in the directory
import glob
csv_files = glob.glob(os.path.join(citibike_path, "*.csv"))

# Filter for 2018 files only (should already be filtered by directory)
csv_files = [f for f in csv_files if "2018" in f]

print(f"\nFound {len(csv_files)} Citi Bike CSV files")
print(f"Sample files: {csv_files[:3]}")

# Load all CSV files and concatenate
print("\nLoading Citi Bike data...")
citibike_dfs = []
for file in csv_files:
    temp_df = pd.read_csv(file)
    citibike_dfs.append(temp_df)
    
citibike_df = pd.concat(citibike_dfs, ignore_index=True)

print(f"\n✓ Loaded Citi Bike data")
print(f"  Shape: {citibike_df.shape}")
print(f"  Columns: {citibike_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(citibike_df.head())

LOADING CITI BIKE DATA

Found 14 Citi Bike CSV files
Sample files: ['../2018-citibike-tripdata/201809-citibike-tripdata.csv', '../2018-citibike-tripdata/201801-citibike-tripdata.csv', '../2018-citibike-tripdata/201803-citibike-tripdata.csv']

Loading Citi Bike data...

✓ Loaded Citi Bike data
  Shape: (18855882, 15)
  Columns: ['tripduration', 'starttime', 'stoptime', 'start station id', 'start station name', 'start station latitude', 'start station longitude', 'end station id', 'end station name', 'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'birth year', 'gender']

First few rows:
   tripduration                 starttime                  stoptime  \
0          1635  2018-09-01 00:00:05.2690  2018-09-01 00:27:20.6340   
1           132  2018-09-01 00:00:11.2810  2018-09-01 00:02:23.4810   
2          3337  2018-09-01 00:00:20.6490  2018-09-01 00:55:58.5470   
3           436  2018-09-01 00:00:21.7460  2018-09-01 00:07:38.5830   
4          8457  2018-09-01 0

### Prepare Citi Bike Data for Merging

Extract merge keys from Citi Bike trip data:
- **`date`**: Extract date from `starttime` column
- **`hour`**: Extract hour from `starttime` column

These keys will match each trip to the corresponding hourly weather data.

In [15]:
# Prepare Citi Bike data for merging with weather data
print("="*60)
print("PREPARING CITI BIKE DATA FOR MERGE")
print("="*60)

# Find the start time column (could be 'starttime', 'start time', 'Start Time', etc.)
time_cols = [col for col in citibike_df.columns if 'start' in col.lower() and 'time' in col.lower()]
if not time_cols:
    time_cols = [col for col in citibike_df.columns if 'time' in col.lower()]

print(f"\nTime column found: {time_cols[0]}")

# Convert start time to datetime
start_time_col = time_cols[0]
citibike_df[start_time_col] = pd.to_datetime(citibike_df[start_time_col])

# Extract date and hour for merging
citibike_df['date'] = citibike_df[start_time_col].dt.date
citibike_df['hour'] = citibike_df[start_time_col].dt.hour

print(f"\n✓ Created 'date' and 'hour' columns for merging")
print(f"  Date range: {citibike_df['date'].min()} to {citibike_df['date'].max()}")
print(f"  Hour range: {citibike_df['hour'].min()} to {citibike_df['hour'].max()}")
print(f"\nSample date-hour combinations:")
print(citibike_df[['date', 'hour']].head(10))

PREPARING CITI BIKE DATA FOR MERGE

Time column found: starttime

✓ Created 'date' and 'hour' columns for merging
  Date range: 2018-01-01 to 2018-12-31
  Hour range: 0 to 23

Sample date-hour combinations:
         date  hour
0  2018-09-01     0
1  2018-09-01     0
2  2018-09-01     0
3  2018-09-01     0
4  2018-09-01     0
5  2018-09-01     0
6  2018-09-01     0
7  2018-09-01     0
8  2018-09-01     0
9  2018-09-01     0


### Merge Citi Bike and Weather Data

Perform a **LEFT JOIN** to merge Citi Bike trips with weather data using `date` and `hour` as keys. This preserves all Citi Bike trips while adding weather columns to each row. The merge adds weather conditions (temperature, precipitation, wind) to each trip based on when it started.

In [16]:
# Merge Citi Bike data with weather data
print("="*60)
print("MERGING CITI BIKE WITH WEATHER DATA")
print("="*60)

# The cleaned weather data is in 'df' variable
# Ensure both date columns are the same type for merging
weather_df = df.copy()

print(f"\nBefore merge:")
print(f"  Citi Bike data: {citibike_df.shape}")
print(f"  Weather data: {weather_df.shape}")

# Merge on both date and hour
merged = citibike_df.merge(
    weather_df,
    on=['date', 'hour'],
    how='left'
)

print(f"\n✓ Merge completed!")
print(f"  Merged dataset shape: {merged.shape}")
print(f"  Merge type: LEFT JOIN (keep all Citi Bike trips)")
print(f"  Merge keys: ['date', 'hour']")

# Check how many rows have weather data
weather_matched = merged['temperature_2m_(°c)'].notna().sum()
print(f"\n  Trips with weather data: {weather_matched:,} ({weather_matched/len(merged)*100:.1f}%)")
print(f"  Trips without weather data: {merged['temperature_2m_(°c)'].isna().sum():,}")

print("\n" + "="*60)

MERGING CITI BIKE WITH WEATHER DATA

Before merge:
  Citi Bike data: (18855882, 17)
  Weather data: (8760, 19)

✓ Merge completed!
  Merged dataset shape: (18855882, 34)
  Merge type: LEFT JOIN (keep all Citi Bike trips)
  Merge keys: ['date', 'hour']

  Trips with weather data: 18,855,882 (100.0%)
  Trips without weather data: 0



### Sanity Check: Verify Merged Dataset Shape

Verify the merge was successful by checking the dataset dimensions. The merged dataset should have:
- Same number of rows as original Citi Bike data
- Combined columns from both datasets

In [17]:
# Sanity Check 1: Check the shape of the merged dataset
print("="*60)
print("MERGED DATASET SHAPE")
print("="*60)
print(f"\nShape of merged dataset: {merged.shape}")
print(f"  Rows: {merged.shape[0]:,}")
print(f"  Columns: {merged.shape[1]}")
print("\n" + "="*60)

MERGED DATASET SHAPE

Shape of merged dataset: (18855882, 34)
  Rows: 18,855,882
  Columns: 34



### Sanity Check: Preview Merged Data

Display a sample of the merged dataset showing key columns (date, hour, temperature, rain flag, wind speed) to verify the weather data was correctly matched to trips.

In [18]:
# Sanity Check 2: Display head of key columns
print("="*60)
print("SAMPLE OF MERGED DATASET")
print("="*60)
print("\nFirst 10 rows of key columns:")
print(merged[["date", "hour", "temperature_2m_(°c)", "rain_flag", "wind_kmh"]].head(10))
print("\n" + "="*60)

SAMPLE OF MERGED DATASET

First 10 rows of key columns:
         date  hour  temperature_2m_(°c)  rain_flag  wind_kmh
0  2018-09-01     0                 22.1          1   16.0934
1  2018-09-01     0                 22.1          1   16.0934
2  2018-09-01     0                 22.1          1   16.0934
3  2018-09-01     0                 22.1          1   16.0934
4  2018-09-01     0                 22.1          1   16.0934
5  2018-09-01     0                 22.1          1   16.0934
6  2018-09-01     0                 22.1          1   16.0934
7  2018-09-01     0                 22.1          1   16.0934
8  2018-09-01     0                 22.1          1   16.0934
9  2018-09-01     0                 22.1          1   16.0934



### Sanity Check: Missing Value Analysis

Check for missing values in key weather columns after the merge. If any trips couldn't be matched with weather data, they will have NaN values. Drop these rows to ensure complete data for modeling.

In [19]:
# Sanity Check 3: Check for missing values in key weather columns
print("="*60)
print("MISSING VALUE ANALYSIS")
print("="*60)
print("\nProportion of missing values in key weather columns:")
missing_props = merged[["temperature_2m_(°c)", "rain_flag", "wind_kmh"]].isna().mean()
print(missing_props)
print(f"\nSummary:")
print(f"  Temperature missing: {missing_props['temperature_2m_(°c)']*100:.2f}%")
print(f"  Rain flag missing: {missing_props['rain_flag']*100:.2f}%")
print(f"  Wind speed missing: {missing_props['wind_kmh']*100:.2f}%")

# Drop rows with missing weather data
before_drop = len(merged)
merged = merged.dropna(subset=["temperature_2m_(°c)", "rain_flag", "wind_kmh"])
after_drop = len(merged)

if before_drop > after_drop:
    print(f"\n✓ Dropped {before_drop - after_drop:,} rows with missing weather data")
    print(f"  Remaining rows: {after_drop:,}")
else:
    print(f"\n✓ No rows with missing weather data - all trips have complete weather information!")

print("\n" + "="*60)

MISSING VALUE ANALYSIS

Proportion of missing values in key weather columns:
temperature_2m_(°c)    0.0
rain_flag              0.0
wind_kmh               0.0
dtype: float64

Summary:
  Temperature missing: 0.00%
  Rain flag missing: 0.00%
  Wind speed missing: 0.00%

✓ No rows with missing weather data - all trips have complete weather information!



## Create Clean Modeling Dataset

Now let's create a clean dataset ready for modeling by adding useful features and filtering out unreasonable trips.

In [20]:
import numpy as np

print("="*60)
print("CREATING MODELING DATASET")
print("="*60)

# Create a copy of the merged dataset
df = merged.copy()

print(f"\nStarting with {len(df):,} trips")

CREATING MODELING DATASET

Starting with 18,855,882 trips


### Initialize Modeling Dataset

Create a working copy of the merged dataset for further feature engineering. This copy will be transformed into the final modeling dataset.

In [21]:
# Step 1: Convert trip duration from seconds to minutes
df["trip_duration_min"] = df["tripduration"] / 60

print("\n" + "="*60)
print("TRIP DURATION")
print("="*60)
print(f"\nTrip duration statistics (in minutes):")
print(f"  Mean: {df['trip_duration_min'].mean():.2f} min")
print(f"  Median: {df['trip_duration_min'].median():.2f} min")
print(f"  Min: {df['trip_duration_min'].min():.2f} min")
print(f"  Max: {df['trip_duration_min'].max():.2f} min")
print(f"  Std: {df['trip_duration_min'].std():.2f} min")


TRIP DURATION

Trip duration statistics (in minutes):
  Mean: 16.46 min
  Median: 10.07 min
  Min: 1.02 min
  Max: 325167.48 min
  Std: 316.62 min


### Convert Trip Duration to Minutes

Convert trip duration from seconds (original format) to minutes for easier interpretation. Display statistics including mean, median, min, max, and standard deviation.

In [22]:
# Step 2: Keep only reasonable trips (1-120 minutes)
print("\n" + "="*60)
print("FILTERING UNREASONABLE TRIPS")
print("="*60)

before_filter = len(df)
df = df[(df["trip_duration_min"] >= 1) & (df["trip_duration_min"] <= 120)]
after_filter = len(df)

print(f"\nFiltered trips outside 1-120 minute range:")
print(f"  Before: {before_filter:,} trips")
print(f"  After: {after_filter:,} trips")
print(f"  Removed: {before_filter - after_filter:,} trips ({(before_filter - after_filter)/before_filter*100:.2f}%)")

print(f"\nNew trip duration statistics (in minutes):")
print(f"  Mean: {df['trip_duration_min'].mean():.2f} min")
print(f"  Median: {df['trip_duration_min'].median():.2f} min")
print(f"  Min: {df['trip_duration_min'].min():.2f} min")
print(f"  Max: {df['trip_duration_min'].max():.2f} min")


FILTERING UNREASONABLE TRIPS

Filtered trips outside 1-120 minute range:
  Before: 18,855,882 trips
  After: 18,799,954 trips
  Removed: 55,928 trips (0.30%)

New trip duration statistics (in minutes):
  Mean: 13.40 min
  Median: 10.03 min
  Min: 1.02 min
  Max: 120.00 min


### Filter Unreasonable Trip Durations

Remove trips with unrealistic durations:
- **Minimum**: 1 minute (trips shorter are likely errors/cancellations)
- **Maximum**: 120 minutes (2 hours - trips longer are statistical outliers)

This removes approximately 0.3% of trips and ensures data quality.

In [23]:
# Step 3: Create log-transformed duration for regression
df["log_duration"] = np.log(df["trip_duration_min"])

print("\n" + "="*60)
print("LOG DURATION")
print("="*60)
print(f"\nLog duration statistics:")
print(f"  Mean: {df['log_duration'].mean():.4f}")
print(f"  Median: {df['log_duration'].median():.4f}")
print(f"  Min: {df['log_duration'].min():.4f}")
print(f"  Max: {df['log_duration'].max():.4f}")
print(f"  Std: {df['log_duration'].std():.4f}")

print(f"\n✓ Created 'log_duration' for regression modeling")


LOG DURATION

Log duration statistics:
  Mean: 2.3111
  Median: 2.3059
  Min: 0.0165
  Max: 4.7875
  Std: 0.7642

✓ Created 'log_duration' for regression modeling


### Create Log-Transformed Duration

Apply natural logarithm transformation to trip duration. Log transformation is useful for:
- Normalizing right-skewed duration distribution
- Improving regression model performance
- Stabilizing variance across different duration ranges

In [24]:
# Step 4: Create weekday and weekend features
print("\n" + "="*60)
print("TIME FEATURES")
print("="*60)

# Extract weekday (0=Monday, 6=Sunday)
df["weekday"] = df["starttime"].dt.weekday

# Create weekend flag (Saturday=5, Sunday=6)
df["is_weekend"] = (df["weekday"] >= 5).astype(int)

print(f"\n✓ Created time-based features:")
print(f"  'weekday': Day of week (0=Monday, 6=Sunday)")
print(f"  'is_weekend': Binary flag for weekend trips")

print(f"\nWeekday distribution:")
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for i, name in enumerate(weekday_names):
    count = (df['weekday'] == i).sum()
    print(f"  {name}: {count:,} trips ({count/len(df)*100:.1f}%)")

print(f"\nWeekend vs Weekday:")
print(f"  Weekday trips: {(df['is_weekend'] == 0).sum():,} ({(df['is_weekend'] == 0).sum()/len(df)*100:.1f}%)")
print(f"  Weekend trips: {(df['is_weekend'] == 1).sum():,} ({(df['is_weekend'] == 1).sum()/len(df)*100:.1f}%)")


TIME FEATURES

✓ Created time-based features:
  'weekday': Day of week (0=Monday, 6=Sunday)
  'is_weekend': Binary flag for weekend trips

Weekday distribution:
  Monday: 2,716,653 trips (14.5%)
  Tuesday: 2,865,435 trips (15.2%)
  Wednesday: 2,899,018 trips (15.4%)
  Thursday: 2,929,111 trips (15.6%)
  Friday: 2,780,371 trips (14.8%)
  Saturday: 2,453,795 trips (13.1%)
  Sunday: 2,155,571 trips (11.5%)

Weekend vs Weekday:
  Weekday trips: 14,190,588 (75.5%)
  Weekend trips: 4,609,366 (24.5%)


### Create Time-Based Features

Extract temporal features from trip start time:
- **`weekday`**: Day of week (0=Monday to 6=Sunday)
- **`is_weekend`**: Binary indicator (1=Saturday/Sunday, 0=weekday)

These features capture weekly patterns in bike usage.

In [25]:
# Summary of the modeling dataset
print("\n" + "="*60)
print("MODELING DATASET SUMMARY")
print("="*60)

print(f"\nFinal dataset shape: {df.shape}")
print(f"  Rows: {df.shape[0]:,}")
print(f"  Columns: {df.shape[1]}")

print(f"\nKey features available:")
print(f"\n  Duration features:")
print(f"    - trip_duration_min: Trip duration in minutes")
print(f"    - log_duration: Log-transformed duration for regression")

print(f"\n  Weather features:")
print(f"    - temperature_2m_(°c): Temperature in Celsius")
print(f"    - rain_flag: Binary indicator for rain")
print(f"    - wind_kmh: Wind speed in km/h")

print(f"\n  Time features:")
print(f"    - date: Date of trip")
print(f"    - hour: Hour of day (0-23)")
print(f"    - weekday: Day of week (0-6)")
print(f"    - is_weekend: Weekend indicator (0/1)")

print(f"\n✓ Dataset is ready for modeling!")
print("=" * 60)


MODELING DATASET SUMMARY

Final dataset shape: (18799954, 38)
  Rows: 18,799,954
  Columns: 38

Key features available:

  Duration features:
    - trip_duration_min: Trip duration in minutes
    - log_duration: Log-transformed duration for regression

  Weather features:
    - temperature_2m_(°c): Temperature in Celsius
    - rain_flag: Binary indicator for rain
    - wind_kmh: Wind speed in km/h

  Time features:
    - date: Date of trip
    - hour: Hour of day (0-23)
    - weekday: Day of week (0-6)
    - is_weekend: Weekend indicator (0/1)

✓ Dataset is ready for modeling!


### Modeling Dataset Summary

Display final summary of the prepared modeling dataset including:
- Dataset dimensions
- Available features grouped by category (duration, weather, time)
- Confirmation that dataset is ready for model training

In [26]:
# Display sample of the modeling dataset
print("\nSample of modeling dataset with key features:")
key_cols = ['date', 'hour', 'weekday', 'is_weekend', 'trip_duration_min', 'log_duration', 
            'temperature_2m_(°c)', 'rain_flag', 'wind_kmh']
# Only select columns that exist
key_cols_existing = [col for col in key_cols if col in df.columns]
print(df[key_cols_existing].head(10))


Sample of modeling dataset with key features:
          date  hour  weekday  is_weekend  trip_duration_min  log_duration  \
0   2018-09-01     0        5           1          27.250000      3.305054   
1   2018-09-01     0        5           1           2.200000      0.788457   
2   2018-09-01     0        5           1          55.616667      4.018483   
3   2018-09-01     0        5           1           7.266667      1.983298   
5   2018-09-01     0        5           1          55.433333      4.015181   
6   2018-09-01     0        5           1           3.883333      1.356694   
7   2018-09-01     0        5           1           4.750000      1.558145   
8   2018-09-01     0        5           1          18.783333      2.932970   
9   2018-09-01     0        5           1           7.050000      1.953028   
10  2018-09-01     0        5           1           5.450000      1.695616   

    temperature_2m_(°c)  rain_flag  wind_kmh  
0                  22.1          1   16.0934  


### Preview Modeling Dataset

Display sample rows of the final modeling dataset showing key feature columns to verify all transformations were applied correctly.

## Save Modeling Dataset

The modeling dataset is now ready. We'll save it for use in the next notebook (scriptp2.ipynb) where we'll build and train the prediction model.

In [27]:
# Save the modeling dataset to CSV
print("="*60)
print("SAVING MODELING DATASET")
print("="*60)

output_filename = "modeling_dataset.csv"

# Save to CSV
df.to_csv(output_filename, index=False)

print(f"\n✓ Modeling dataset saved successfully!")
print(f"  Filename: {output_filename}")
print(f"  Location: {os.getcwd()}")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {len(df.columns)}")

print(f"\n  Key columns saved:")
print(f"    - Trip data: tripduration, trip_duration_min, log_duration")
print(f"    - Weather: temperature_2m_(°c), rain_flag, wind_kmh")
print(f"    - Time: date, hour, weekday, is_weekend")

print("\n" + "="*60)
print("✅ DATA CLEANING COMPLETE!")
print("="*60)
print("\nNext steps:")
print("  → Open 'scriptp2.ipynb' to build and train the prediction model")
print("="*60)

SAVING MODELING DATASET

✓ Modeling dataset saved successfully!
  Filename: modeling_dataset.csv
  Location: /Users/hstro/Documents/DTU/1st Semester/Introduction to business analytics/Project/Part 2
  Rows: 18,799,954
  Columns: 38

  Key columns saved:
    - Trip data: tripduration, trip_duration_min, log_duration
    - Weather: temperature_2m_(°c), rain_flag, wind_kmh
    - Time: date, hour, weekday, is_weekend

✅ DATA CLEANING COMPLETE!

Next steps:
  → Open 'scriptp2.ipynb' to build and train the prediction model
